In [82]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

# CONSTS

In [83]:
CHROME_DRIVER_PATH = "./chromedriver-linux/chromedriver"
BRAVE_PATH = "/usr/bin/brave-browser"
DOWNLOAD_DIRECTORY = "/home/etudiant/dossier_partage/donnees-meteo/"
ARCHIVE_DIRECTORY = "/home/etudiant/dossier_partage/donnees-meteo/archive"
URL = "https://donneespubliques.meteofrance.fr/?fond=produit&id_produit=90&id_rubrique=32"

## Browser Configs

In [84]:
options = Options()
options.binary_location = BRAVE_PATH  

prefs = {
    "download.default_directory": DOWNLOAD_DIRECTORY,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True 
}
options.add_experimental_option("prefs", prefs)

In [85]:
service = Service(executable_path=CHROME_DRIVER_PATH)

driver = webdriver.Chrome(service=service, options=options)



driver.get(URL)
print("Page Title:", driver.title)
telecharger_button = driver.find_element(By.XPATH, "//input[@type='submit' and @value='Télécharger']")
telecharger_button.click()

time.sleep(5)
driver.quit()

Page Title: Données Publiques de Météo-France - Données SYNOP essentielles OMM


## Fixing the file extension problem

In [86]:
from file_management import change_file_extension

cvs_file = change_file_extension(DOWNLOAD_DIRECTORY)

Le fichier a été renommé : /home/etudiant/dossier_partage/donnees-meteo/synop.2025012012.txt -> /home/etudiant/dossier_partage/donnees-meteo/synop.2025012012.csv


# Starting the ELT

In [87]:
import pandas as pd
import numpy as np
df = pd.read_csv(cvs_file, sep=";", dtype={"date": str})

print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww  w1  w2    n nbas  hbas  cl  cm  ch    pres niv_bar geop tend24        tn12        tn24 tx12        tx24     tminsol  sw  tw      raf10     rafper  per etat_sol  ht_neige    ssfrai perssfrai        rr1        rr3        rr6       rr12       rr24 phenspe1 phenspe2 phenspe3 phenspe4 nnuage1 ctype1 hnuage1 nnuage2 ctype2 hnuage2 nnuage3 ctype3 hnuage3 nnuage4 ctype4 hnuage4  Unnamed: 59
0        7005  20250120120000  101990   -50        8  140   2.0  276.450000  273.650000   82   8480   0  mq  mq   mq   mq    mq  mq  mq  mq  101090      mq   mq   -240          mq          mq   mq          mq  273.150000  mq  mq   3.900000   5.600000  -10        0  0.000000        mq        mq   0.000000   0.000000   0.000000   0.000000   0.000000       mq       mq       mq       mq      mq     mq      mq      mq     mq      mq      mq     mq      mq      mq     mq      mq          NaN
1        7015  202

# Récupération des informations importantes à savoir :
1. Indicatif OMM station : numéro de station `numer_sta` -> `int`
2. Date (UTC) : `date` -> `datetime`
3. Pression au niveau mer : `pmer` -> `int`
4. Variation de pression en 3 heures : `tend` -> `int`
5. Type de tendance barométrique : `cod_tend` -> `int`
6. Direction du vent moyen 10 mn : `dd` -> `int`
7. Vitesse du vent moyen 10 mn : `ff` -> `float`
8. Température : `t` -> `float`
9. Point de rosée : `td` -> `float`
10. Humidité : `u` -> `int`
11. Visibilité horizontale : `vv` -> `float`
12. Temps présent : `ww` -> `int`
13. Nébulosité totale : `n` -> `float`
14. Nébulosité des nuages de l'étage inférieur : `nbas` -> `int`
15. Hauteur de la base des nuages de l'étage inférieur : `hbas` -> `int`
16. Pression station : `pres` -> `int`
17. Variation de pression en 24 heures : `tend24` -> `int`
18. Température minimale sur N heures : `tn12` -> `float`
19. Température maximale sur N heures : `tx12` -> `float`
20. Température minimale du sol sur 12 heures : `tminsol` -> `float`
21. Rafales sur les 10 dernières minutes : `raf10` -> `float`
22. Rafales sur une période : `rafper` -> `float`
23. Période de mesure des rafales : `per` -> `float`
24. Précipitations dans les N dernières heures : `rr12` -> `float`

In [88]:
df = df[['numer_sta', 'date', "pmer", "tend", "cod_tend", "dd", "ff", "t", "td", "u", "vv", "ww", "n", "nbas", "hbas","pres", "tend24", "tn12", "tx12", "tminsol", "raf10", "rafper", "per", "rr12" ]]


print(df.to_string())


    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv  ww    n nbas  hbas    pres tend24        tn12 tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250120120000  101990   -50        8  140   2.0  276.450000  273.650000   82   8480   0   mq   mq    mq  101090   -240          mq   mq  273.150000   3.900000   5.600000  -10   0.000000
1        7015  20250120120000  102020   -40        8  150   2.6  273.350000  273.350000  100    640  45  101    9    30  101420   -280          mq   mq  273.250000   3.700000   3.700000  -10   0.000000
2        7020  20250120120000  101920    70        1  100   3.9  277.950000  275.850000   86     mq  mq   mq   mq    mq  101810    -60          mq   mq          mq   6.500000   7.900000  -10   0.000000
3        7027  20250120120000  101930   -20        8  130   5.0  275.350000  273.850000   90   6840   0  100    8   250  101090   -240          mq   mq  275.350000   6.900000   8.100000  -10  

## Remplacer les valeurs manquantes `mq` par `NaN`

In [89]:
df.replace("mq", np.nan, inplace=True)
print(df.to_string())

    numer_sta            date    pmer  tend cod_tend   dd    ff           t          td    u     vv   ww    n nbas  hbas    pres tend24        tn12  tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250120120000  101990   -50        8  140   2.0  276.450000  273.650000   82   8480    0  NaN  NaN   NaN  101090   -240         NaN   NaN  273.150000   3.900000   5.600000  -10   0.000000
1        7015  20250120120000  102020   -40        8  150   2.6  273.350000  273.350000  100    640   45  101    9    30  101420   -280         NaN   NaN  273.250000   3.700000   3.700000  -10   0.000000
2        7020  20250120120000  101920    70        1  100   3.9  277.950000  275.850000   86    NaN  NaN  NaN  NaN   NaN  101810    -60         NaN   NaN         NaN   6.500000   7.900000  -10   0.000000
3        7027  20250120120000  101930   -20        8  130   5.0  275.350000  273.850000   90   6840    0  100    8   250  101090   -240         NaN   NaN  275.350000   6.900000   8.100

/tmp/ipykernel_1556173/3864656540.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace("mq", np.nan, inplace=True)


## Nettoyages de colonnes non renseignées 

1. Informations générales
	-	Numéro de station : `numer_sta` -> int
	-	Date (UTC) : `date` -> datetime
2. Pression
	-	Pression au niveau mer : `pmer` -> int
	-	Variation de pression en 3 heures : `tend` -> int
	-	Type de tendance barométrique : `cod_tend` -> int
	-	Pression station : `pres` -> int
	-	Variation de pression en 24 heures : `tend24` -> int
3. Vent
	-	Direction du vent moyen 10 mn : `dd` -> int
	-	Vitesse du vent moyen 10 mn : `ff` -> float
	-	Rafales sur les 10 dernières minutes : `raf10` -> float
	-	Rafales sur une période : `rafper` -> float
	-	Période de mesure des rafales : `per` -> float
4. Température
	-	Température : `t` -> float
	-	Point de rosée : `td` -> float
	-	Température minimale sur N heures : `tn12` -> float
	-	Température maximale sur N heures : `tx12` -> float
	-	Température minimale du sol sur 12 heures : `tminsol` -> float
5. Humidité
	-	Humidité : `u` -> int
6. Visibilité
	-	Visibilité horizontale : `vv` -> float
7. Nébulosité et nuages
	-	Nébulosité totale : `n` -> float
	-	Nébulosité des nuages de l’étage inférieur : `nbas` -> int
	-	Hauteur de la base des nuages de l’étage inférieur : `hbas` -> int
8. Précipitations
	-	Précipitations dans les N dernières heures : `rr12` -> float
9. Temps présent
	-	Temps présent : `ww` -> int

In [90]:
df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]] = df[['dd', 'ff', 'raf10', 'rafper', 'rr12', "per"]].fillna(0)

moyenne = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]
df[moyenne] = df[moyenne].apply(pd.to_numeric, errors='coerce')

for col in moyenne:
    df[col] = df[col].fillna(df[col].mean())

print(df.to_string())



    numer_sta            date           pmer        tend  cod_tend   dd    ff           t          td           u            vv         ww           n      nbas         hbas           pres       tend24    tn12  tx12     tminsol      raf10     rafper  per       rr12
0        7005  20250120120000  101990.000000  -50.000000  8.000000  140   2.0  276.450000  273.650000   82.000000   8480.000000   0.000000   93.366667  5.538462   944.285714  101090.000000  -240.000000  296.45   NaN  273.150000   3.900000   5.600000  -10   0.000000
1        7015  20250120120000  102020.000000  -40.000000  8.000000  150   2.6  273.350000  273.350000  100.000000    640.000000  45.000000  101.000000  9.000000    30.000000  101420.000000  -280.000000  296.45   NaN  273.250000   3.700000   3.700000  -10   0.000000
2        7020  20250120120000  101920.000000   70.000000  1.000000  100   3.9  277.950000  275.850000   86.000000  17234.883721   8.790698   93.366667  5.538462   944.285714  101810.000000   -60.000000 

## Typages des colonnes

In [91]:
df["date"] = pd.to_datetime(df["date"], format='%Y%m%d%H%M%S')
df['date'] = df['date'].dt.strftime("%Y-%m-%d %H:%M:%S")

entier = ["numer_sta", "pmer", "tend", "cod_tend", "dd", "u", "ww", "nbas", "hbas", "pres", "tend24"]
for i in entier:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(int)
    print(f"{i} -> {df[i].dtype}")
    print("")

double = ['dd', 'ff', 'raf10', 'rafper', 'rr12', "per", "t", "td", "tn12", "tx12", "tminsol", "n", "vv"]
for i in double:
    print(f"{i} -> {df[i].dtype}")
    df[i] = df[i].astype(float)
    print(f"{i} -> {df[i].dtype}")
    print("")

numer_sta -> int64
numer_sta -> int64

pmer -> float64
pmer -> int64

tend -> float64
tend -> int64

cod_tend -> float64
cod_tend -> int64

dd -> int64
dd -> int64

u -> float64
u -> int64

ww -> float64
ww -> int64

nbas -> float64
nbas -> int64

hbas -> float64
hbas -> int64

pres -> float64
pres -> int64

tend24 -> float64
tend24 -> int64

dd -> int64
dd -> float64

ff -> float64
ff -> float64

raf10 -> object
raf10 -> float64

rafper -> object
rafper -> float64

rr12 -> object
rr12 -> float64

per -> object
per -> float64

t -> float64
t -> float64

td -> float64
td -> float64

tn12 -> float64
tn12 -> float64

tx12 -> float64
tx12 -> float64

tminsol -> float64
tminsol -> float64

n -> float64
n -> float64

vv -> float64
vv -> float64



In [92]:
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff           t          td    u            vv  ww           n  nbas  hbas    pres  tend24    tn12  tx12     tminsol  raf10  rafper   per  rr12
0        7005  2025-01-20 12:00:00  101990   -50         8  140.0   2.0  276.450000  273.650000   82   8480.000000   0   93.366667     5   944  101090    -240  296.45   NaN  273.150000    3.9     5.6 -10.0   0.0
1        7015  2025-01-20 12:00:00  102020   -40         8  150.0   2.6  273.350000  273.350000  100    640.000000  45  101.000000     9    30  101420    -280  296.45   NaN  273.250000    3.7     3.7 -10.0   0.0
2        7020  2025-01-20 12:00:00  101920    70         1  100.0   3.9  277.950000  275.850000   86  17234.883721   8   93.366667     5   944  101810     -60  296.45   NaN  275.067143    6.5     7.9 -10.0   0.0
3        7027  2025-01-20 12:00:00  101930   -20         8  130.0   5.0  275.350000  273.850000   90   6840.000000   0  100.000000     8   250  101090  

## Conversion des températures de Kelvin en Celcuis

In [93]:
celcuis = ["t", "td", "tn12", "tx12", "tminsol"]
for i in celcuis:
    df[i] = df[i] - 273.15

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff          t         td    u            vv  ww           n  nbas  hbas    pres  tend24  tn12  tx12    tminsol  raf10  rafper   per  rr12
0        7005  2025-01-20 12:00:00  101990   -50         8  140.0   2.0   3.300000   0.500000   82   8480.000000   0   93.366667     5   944  101090    -240  23.3   NaN   0.000000    3.9     5.6 -10.0   0.0
1        7015  2025-01-20 12:00:00  102020   -40         8  150.0   2.6   0.200000   0.200000  100    640.000000  45  101.000000     9    30  101420    -280  23.3   NaN   0.100000    3.7     3.7 -10.0   0.0
2        7020  2025-01-20 12:00:00  101920    70         1  100.0   3.9   4.800000   2.700000   86  17234.883721   8   93.366667     5   944  101810     -60  23.3   NaN   1.917143    6.5     7.9 -10.0   0.0
3        7027  2025-01-20 12:00:00  101930   -20         8  130.0   5.0   2.200000   0.700000   90   6840.000000   0  100.000000     8   250  101090    -240  23.3   NaN   2

## Arrondir tous les decimaux à 2chiffres après la virgules

In [94]:
df[double] = df[double].round(2)
print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff     t    td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2025-01-20 12:00:00  101990   -50         8  140.0   2.0   3.3   0.5   82   8480.00   0   93.37     5   944  101090    -240  23.3   NaN     0.00    3.9     5.6 -10.0   0.0
1        7015  2025-01-20 12:00:00  102020   -40         8  150.0   2.6   0.2   0.2  100    640.00  45  101.00     9    30  101420    -280  23.3   NaN     0.10    3.7     3.7 -10.0   0.0
2        7020  2025-01-20 12:00:00  101920    70         1  100.0   3.9   4.8   2.7   86  17234.88   8   93.37     5   944  101810     -60  23.3   NaN     1.92    6.5     7.9 -10.0   0.0
3        7027  2025-01-20 12:00:00  101930   -20         8  130.0   5.0   2.2   0.7   90   6840.00   0  100.00     8   250  101090    -240  23.3   NaN     2.20    6.9     8.1 -10.0   0.0
4        7037  2025-01-20 12:00:00  102000   -30         8  130.0

# Add verif to check is all values

In [95]:
verif = ["pmer", "tend", "cod_tend", "pres", "tend24", "t", "td", "tn12", "tx12", "tminsol", "u", "vv", "n", "nbas", "hbas", "ww"]

for col in verif:
    if df[col].isna().all() :
        df[col].fillna(0.0,inplace=True)

print(df.to_string())

    numer_sta                 date    pmer  tend  cod_tend     dd    ff     t    td    u        vv  ww       n  nbas  hbas    pres  tend24  tn12  tx12  tminsol  raf10  rafper   per  rr12
0        7005  2025-01-20 12:00:00  101990   -50         8  140.0   2.0   3.3   0.5   82   8480.00   0   93.37     5   944  101090    -240  23.3   0.0     0.00    3.9     5.6 -10.0   0.0
1        7015  2025-01-20 12:00:00  102020   -40         8  150.0   2.6   0.2   0.2  100    640.00  45  101.00     9    30  101420    -280  23.3   0.0     0.10    3.7     3.7 -10.0   0.0
2        7020  2025-01-20 12:00:00  101920    70         1  100.0   3.9   4.8   2.7   86  17234.88   8   93.37     5   944  101810     -60  23.3   0.0     1.92    6.5     7.9 -10.0   0.0
3        7027  2025-01-20 12:00:00  101930   -20         8  130.0   5.0   2.2   0.7   90   6840.00   0  100.00     8   250  101090    -240  23.3   0.0     2.20    6.9     8.1 -10.0   0.0
4        7037  2025-01-20 12:00:00  102000   -30         8  130.0

/tmp/ipykernel_1556173/555783230.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(0.0,inplace=True)


# Archive file

In [96]:
from file_management import move_file

move_file(cvs_file, ARCHIVE_DIRECTORY)

Le fichier a été déplacé de /home/etudiant/dossier_partage/donnees-meteo/synop.2025012012.csv à /home/etudiant/dossier_partage/donnees-meteo/archive/synop.2025012012.csv


# Sending data to API

In [97]:
import requests
from api import send_data

In [98]:
API_URL = "http://127.0.0.1:8080/api/donnees-climatiques" 

dict_data = df.to_dict(orient="records")

for row in dict_data:
    print(row)
    send_data(API_URL, row, requests)

{'numer_sta': 7005, 'date': '2025-01-20 12:00:00', 'pmer': 101990, 'tend': -50, 'cod_tend': 8, 'dd': 140.0, 'ff': 2.0, 't': 3.3, 'td': 0.5, 'u': 82, 'vv': 8480.0, 'ww': 0, 'n': 93.37, 'nbas': 5, 'hbas': 944, 'pres': 101090, 'tend24': -240, 'tn12': 23.3, 'tx12': 0.0, 'tminsol': 0.0, 'raf10': 3.9, 'rafper': 5.6, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7015, 'date': '2025-01-20 12:00:00', 'pmer': 102020, 'tend': -40, 'cod_tend': 8, 'dd': 150.0, 'ff': 2.6, 't': 0.2, 'td': 0.2, 'u': 100, 'vv': 640.0, 'ww': 45, 'n': 101.0, 'nbas': 9, 'hbas': 30, 'pres': 101420, 'tend24': -280, 'tn12': 23.3, 'tx12': 0.0, 'tminsol': 0.1, 'raf10': 3.7, 'rafper': 3.7, 'per': -10.0, 'rr12': 0.0}
Données ajoutées avec succès!
{'numer_sta': 7020, 'date': '2025-01-20 12:00:00', 'pmer': 101920, 'tend': 70, 'cod_tend': 1, 'dd': 100.0, 'ff': 3.9, 't': 4.8, 'td': 2.7, 'u': 86, 'vv': 17234.88, 'ww': 8, 'n': 93.37, 'nbas': 5, 'hbas': 944, 'pres': 101810, 'tend24': -60, 'tn12': 23.3, 'tx12': 